### Edgar Moises Hernandez-Gonzalez
#### 21/07/20
#### Clasificacion de imagenes motoras en EEG con CNN-2D + LSTM
##### Caracteristicas = Espectrogramas STFT (3D)
##### Entrada de 135x31

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import time
from scipy.signal import spectrogram
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import (TimeDistributed, Conv2D, MaxPool2D, Flatten, LSTM,
                          Dense, Dropout)
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

Using TensorFlow backend.


In [4]:
# para que este script funcione debe de decir 'channels_last', si dice 'channels_first' no sirve
K.image_data_format()

'channels_last'

In [5]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B5T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B5E.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/etiquetas_train_5.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/etiquetas_test_5.csv", header=None)

In [6]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(420, 3000)
(320, 3000)
(420, 1)
(320, 1)


In [7]:
x = pd.concat([x_train, x_test])
y = pd.concat([y_train, y_test])

In [8]:
print(x.shape)
print(y.shape)

(740, 3000)
(740, 1)


In [9]:
# en caso de no tener el y_test se puede utilizar el x_test
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 740


In [10]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y))

print("n_classes:", n_classes)

n_classes: 2


In [11]:
#unir 3 canales y resize de tamaño proporcional
def unir_espectrogramas_vertical(data):
  fs = 250
  datos = np.zeros((data.shape[0],135,31))
  temporal = np.zeros((135,31))
  for i in range(data.shape[0]): #n muestras
    for j in range(3): #3 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      #sig = sig[124:624]
      #espectrograma de STFFT
      f, t, Sxx = spectrogram(sig, fs=250, window='hann', nperseg=250,
                              noverlap=225, nfft=500, scaling='spectrum')
      temporal[j*45:(j+1)*45, :] = Sxx[16:61, :]
    datos[i] = temporal
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [12]:
#llamar a unir_espectrogramas_vertical_resize(data)
inicio = time.time()

x = unir_espectrogramas_vertical(x)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
400
500
600
700
Tiempo: 2.0443427562713623


In [13]:
print(x.shape)

(740, 135, 31)


In [14]:
print(np.max(x))
print(np.min(x))
print(np.mean(x))

11.889219138973978
1.2170156911484715e-09
0.04855036105545974


In [15]:
#convertir a float
x = x.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x /= np.ceil(np.max(x))

In [16]:
print(np.max(x))
print(np.min(x))
print(np.mean(x))

0.99076825
1.0141798e-10
0.0040458627


In [17]:
# convertir de 3D a 5D
x = x.reshape((x.shape[0], 1, x.shape[1], x.shape[2], 1))

print(x.shape)

(740, 1, 135, 31, 1)


In [18]:
def CNN_LSTM(n_unidades, n_neuronas):
  model = Sequential()
  model.add(TimeDistributed(Conv2D(4, (3,3), activation='relu', padding='same'),
                            input_shape=x.shape[1:]))
  model.add(TimeDistributed(MaxPool2D((2,2))))
  model.add(TimeDistributed(Conv2D(4, (3,3), activation='relu', padding='same')))
  model.add(TimeDistributed(MaxPool2D((2,2))))
  model.add(TimeDistributed(Flatten()))
  model.add(LSTM(n_unidades, activation='tanh', dropout=0.5))
  model.add(Dense(n_neuronas, activation='relu'))
  model.add(Dense(n_classes, activation='softmax'))

  optimizer = Adam(lr=1e-4)
  model.compile(optimizer = optimizer,
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [19]:
clf = KerasClassifier(build_fn = CNN_LSTM, epochs=400, batch_size=36, verbose=0)

In [20]:
n_unidades = [4, 8]
n_neuronas = [16, 32, 64, 128]
hiperparametros = dict(n_unidades = n_unidades,
                       n_neuronas = n_neuronas)

In [21]:
# 8 redes = 634 seg
2*4

8

In [22]:
grid = GridSearchCV(clf, hiperparametros, cv=2)

In [23]:
inicio = time.time()

grid.fit(x, y)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 665.7298510074615


In [24]:
print(grid.best_params_)
print(grid.best_score_)

{'n_neuronas': 16, 'n_unidades': 4}
0.7567567825317383


In [25]:
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

0.757 (+/-0.157) for {'n_neuronas': 16, 'n_unidades': 4}
0.742 (+/-0.132) for {'n_neuronas': 16, 'n_unidades': 8}
0.754 (+/-0.146) for {'n_neuronas': 32, 'n_unidades': 4}
0.746 (+/-0.135) for {'n_neuronas': 32, 'n_unidades': 8}
0.742 (+/-0.132) for {'n_neuronas': 64, 'n_unidades': 4}
0.738 (+/-0.124) for {'n_neuronas': 64, 'n_unidades': 8}
0.749 (+/-0.135) for {'n_neuronas': 128, 'n_unidades': 4}
0.750 (+/-0.138) for {'n_neuronas': 128, 'n_unidades': 8}


In [26]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jul 21 23:16:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    28W /  70W |    805MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------